In [86]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time, os
from datetime import datetime
import random
import pickle
from time import sleep
from collections import defaultdict

In [87]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [103]:
with open('pickles/cleaned_210115_data.pickle','rb') as read_file:
    df = pickle.load(read_file)

*** thoughts .. maybe find walk score for neighborhood or zip code instead of address

In [4]:
city_list = list(df.city.unique())
city_list[:3]

['Oakland', 'Emeryville', 'San Leandro']

In [5]:
city_tags = [city.replace(' ', '_') for city in city_list]
base_url = 'https://www.walkscore.com/CA/'
urls = [base_url + city_tag for city_tag in city_tags]
city_dict = dict(zip(urls, city_list))
# response = requests.get(url)
# print(response)
# search_page = response.text
# search_page_soup = BeautifulSoup(search_page, 'lxml')

In [40]:
idx = city_list.index('Pleasanton')

In [41]:
idx

9

In [47]:
try:
    int('a')
except:
    print('cant do it')

cant do it


In [66]:
url = list(city_dict.keys())[4]
city = city_dict[url]
print(url, city)

https://www.walkscore.com/CA/Castro_Valley Castro Valley


In [62]:
def assemble_score_dict(neighborhood_val, score_val):
    scores['city'].append(city_dict[url])
    scores['neighborhood'].append(neighborhood_val)
    scores['walk'].append(score_val)

In [70]:
scores = defaultdict(list)
for url in urls[2:5]:    
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)

    time.sleep(4)
    try:
        button = driver.find_element_by_id('read-more-link')
        button.click()
    except:
        print('button not found')

    time.sleep(4)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    driver.quit()

    all_text = soup.text.lower()
    
    try:
        for i in soup.find('tbody'):
            neighborhood_name = i.find('a')
            if neighborhood_name != -1:
                assemble_score_dict(neighborhood_name.text, neighborhood_name.findNext().text)
    except TypeError:
        try:
            ws_idx = all_text.find('has an average walk score of ') + len('has an average walk score of ')
            avg_score = all_text[ws_idx: ws_idx+2]
            assemble_score_dict('N/A', avg_score)

        except:
            assemble_score_dict('N/A', 'N/A')

button not found


In [67]:
    scores = defaultdict(list)

    all_text = soup.text.lower()
    try:
        for i in soup.find('tbody'):
            neighborhood_name = i.find('a')
            if neighborhood_name != -1:
                assemble_score_dict(neighborhood_name.text, neighborhood_name.findNext().text)
    except TypeError:
        try:
            ws_idx = all_text.find('has an average walk score of ') + len('has an average walk score of ')
            avg_score = all_text[ws_idx: ws_idx+2]
            assemble_score_dict('N/A', avg_score)

        except:
            assemble_score_dict('N/A', 'N/A')

walk_data = pd.DataFrame.from_dict(scores)

In [83]:
walk_data = pd.DataFrame.from_dict(scores)
print(walk_data.city.unique())
walk_data[walk_data.city == 'Berkeley']

['San Leandro' 'Berkeley' 'Castro Valley']


,city,neighborhood,walk
18,Berkeley,Downtown,97
19,Berkeley,Southside,95
20,Berkeley,North Berkeley,91
21,Berkeley,South Berkeley,89
22,Berkeley,Central Berkeley,88
23,Berkeley,West Berkeley,87
24,Berkeley,Northside,85
25,Berkeley,Northbrae,84
26,Berkeley,Thousand Oaks,81
27,Berkeley,Panoramic Hill,66


---

getting walk scores by neighborhood doesn't work since walkscore.com uses different neighborhood designations than compass.com

reverting to the individual address walk score idea instead

---

In [134]:
street_type = {'road':'rd', 'real':'real', 'boulevard': 'blvd', 'court': 'ct', 'avenue': 'ave', 'lane': 'ln', 'street': 'st', 'drive': 'dr', 
               'way': 'way', 'parkway':'pkwy', 'common':'common', 'parks': 'parks', 'terrace': 'terrace', 'st.': 'st',
              'place': 'pl', 'circle': 'cir', 'loop':'loop', 'estates': 'estates', 'plaza': 'plaza', 'chavez': 'chavez',
              'broadway':'broadway', 'park':'parks', 'east': 'ave-e', 'alley': 'alley', 'shipley':'shipley', 'mar':'mar',
              'prenda': 'prenda'}
addresses = [address.lower().split(' ') for address in list(df.address_1)]
try:    
    for address in addresses:
        address[-1] = street_type[address[-1]]
except KeyError:
    pass
url_pt1 = 'https://www.walkscore.com/score/'
urls_pt2 = [('-').join(address) for address in addresses]
urls_pt3 = ['-'+city.lower().replace(' ', '-')+'-ca-'+zip_code for city, zip_code in zip(list(df.city), list(df.zip))]
urls = [url_pt1 + urls_pt2[i] + urls_pt3[i] for i in range(0, len(urls_pt2))]

In [97]:
df[df.address_1 == '3934 Cesar Chavez']

,city,zip,neighborhood,address_1,address_2,price,beds,baths,half_baths,sq_footage,...,mls_type,year_built,county,laundry_type,laundry_snippet,parking_spots,parking_snippet,has_address_2,baths_incl_half,clean_mls_types
271,San Francisco,94131,Noe Valley,3934 Cesar Chavez,N/A,6000.0,3.0,2.0,0.0,1410,...,LEASE RENTAL / Apartment,1900,San Francisco County,0,none,parking_unknown,,0,2.0,apartment
